In [2]:
import pickle

n_reps = 4

filepath = f"n2_sto-6g_10e8o_{n_reps}_gradient.pickle"
with open(filepath, "rb") as f:
    data_gradient = pickle.load(f)


filepath = f"n2_sto-6g_10e8o_{n_reps}_gradient-t2_dagger.pickle"
with open(filepath, "rb") as f:
    data_gradient_dagger = pickle.load(f)

filepath = f"n2_sto-6g_10e8o_{n_reps}_gradient_multi_stage.pickle"
with open(filepath, "rb") as f:
    data_gradient_multi_stage = pickle.load(f)

In [3]:
print("energy")
print(f"data_gradient: {data_gradient['energy']}")
print(f"data_gradient_dagger: {data_gradient_dagger['energy']}")
print(f"data_gradient_gradient_multi_stage: {data_gradient_multi_stage['energy']}")

energy
data_gradient: -108.575632043211
data_gradient_dagger: -108.4639160716306
data_gradient_gradient_multi_stage: -106.39931406103938


In [4]:
print("error")
print(f"data_gradient: {data_gradient['error']}")
print(f"data_gradient_dagger: {data_gradient_dagger['error']}")
print(f"data_gradient_gradient_multi_stage: {data_gradient_multi_stage['error']}")

error
data_gradient: 0.02035530776394978
data_gradient_dagger: 0.13207127934434482
data_gradient_gradient_multi_stage: 2.1966732899355748


In [5]:
print("final_loss")
print(f"data_gradient: {data_gradient['final_loss']}")
print(f"data_gradient_dagger: {data_gradient_dagger['final_loss']}")
print(f"data_gradient_gradient_multi_stage: {data_gradient_multi_stage['final_loss']}")

final_loss
data_gradient: 0.0038108772820247333
data_gradient_dagger: 0.05763456970453262
data_gradient_gradient_multi_stage: 5.9841466281795874e-05


In [6]:
operator_gradient = data_gradient["operator"]
operator_gradient_dagger = data_gradient_dagger["operator"]
operator_gradient_multi_stage = data_gradient_multi_stage["operator"]

In [7]:
from opt_einsum import contract

nocc = 5


def fun(diag_coulomb_mats, orbital_rotations):
    reconstructed = (
        1j
        * contract(
            "mpq,map,mip,mbq,mjq->ijab",
            diag_coulomb_mats,
            orbital_rotations,
            orbital_rotations.conj(),
            orbital_rotations,
            orbital_rotations.conj(),
            # optimize="greedy"
        )[:nocc, :nocc, nocc:, nocc:]
    )
    return reconstructed

In [2]:
molecule_name = "n2"
basis = "sto-6g"
nelectron, norb = 10, 8


molecule_basename = f"{molecule_name}_{basis}_{nelectron}e{norb}o"

bond_distance = 1.0

from molecules_catalog.util import load_molecular_data
from pathlib import Path
import os
from ffsim.variational.util import interaction_pairs_spin_balanced

# Get molecular data and molecular Hamiltonian
molecules_catalog_dir = "../../molecules-catalog"

mol_data = load_molecular_data(
    f"{molecule_basename}_d-{bond_distance:.5f}",
    molecules_catalog_dir=molecules_catalog_dir,
)
norb = mol_data.norb
nelec = mol_data.nelec
t2 = mol_data.ccsd_t2
print(nelec)

(5, 5)


In [9]:
import numpy as np

diag_coulomb_mats_gradient, _ = np.unstack(operator_gradient.diag_coulomb_mats, axis=1)
reconstrcuted_operator_gradient = fun(
    diag_coulomb_mats_gradient, operator_gradient.orbital_rotations
)

print(f"t2 norm: {np.linalg.norm(t2)}")
print()
print(np.linalg.norm(reconstrcuted_operator_gradient))
print(np.max(np.abs((reconstrcuted_operator_gradient))))
print()
diag_coulomb_mats_gradient_multi_stage, _ = np.unstack(
    operator_gradient_multi_stage.diag_coulomb_mats, axis=1
)
reconstrcuted_operator_gradient_multi_stage = fun(
    diag_coulomb_mats_gradient_multi_stage,
    operator_gradient_multi_stage.orbital_rotations,
)

print(np.linalg.norm(reconstrcuted_operator_gradient_multi_stage))
print(np.max(np.abs((reconstrcuted_operator_gradient_multi_stage))))
diff = reconstrcuted_operator_gradient_multi_stage - t2
loss = 0.5 * np.sum(np.abs(diff) ** 2)
print(f"loss: {loss}")

t2 norm: 0.2325443258525995

0.21553559262736588
0.11822834762201273

0.23212473501833156
0.11462342079201747
loss: 5.9841298593169374e-05


In [11]:
# Compute final state
import ffsim

reference_state = ffsim.hartree_fock_state(norb, nelec)
final_state_gradient = ffsim.apply_unitary(
    reference_state, operator_gradient, norb=norb, nelec=nelec
)
final_state_gradient_dagger = ffsim.apply_unitary(
    reference_state, operator_gradient_dagger, norb=norb, nelec=nelec
)
final_state_multi_stage = ffsim.apply_unitary(
    reference_state, operator_gradient_multi_stage, norb=norb, nelec=nelec
)

In [14]:
# Run SQD
from qiskit.primitives import BitArray
from qiskit_addon_sqd.fermion import diagonalize_fermionic_hamiltonian, solve_sci_batch

shots = 100_000
samples_per_batch = 100
n_batches = 3
energy_tol = 1e-5
occupancies_tol = 1e-3
max_iterations = 100
symmetrize_spin = True
carryover_threshold = 1e-3
entropy = 0

mol_ham = mol_data.hamiltonian

final_states = [
    final_state_gradient,
    final_state_gradient_dagger,
    final_state_multi_stage,
]
names = ["gradient", "gradient_dagger", "multi_stage"]
for final_state, name in zip(final_states, names):
    rng = np.random.default_rng(entropy)
    samples = ffsim.sample_state_vector(
        final_state,
        norb=norb,
        nelec=nelec,
        shots=shots,
        seed=rng,
        bitstring_type=ffsim.BitstringType.INT,
    )
    bit_array = BitArray.from_samples(samples, num_bits=2 * norb)
    result = diagonalize_fermionic_hamiltonian(
        mol_ham.one_body_tensor,
        mol_ham.two_body_tensor,
        bit_array,
        samples_per_batch=samples_per_batch,
        norb=norb,
        nelec=nelec,
        num_batches=n_batches,
        energy_tol=energy_tol,
        occupancies_tol=occupancies_tol,
        max_iterations=max_iterations,
        sci_solver=solve_sci_batch,
        symmetrize_spin=symmetrize_spin,
        carryover_threshold=carryover_threshold,
        seed=rng,
    )
    energy = result.energy + mol_data.core_energy
    sci_state = result.sci_state
    spin_squared = sci_state.spin_square()
    error = energy - mol_data.fci_energy
    print(f"sqd energy loss from {name}: {error}")

sqd energy loss from gradient: 0.01234564509971392
sqd energy loss from gradient_dagger: 0.018334499964836937
sqd energy loss from multi_stage: 3.355100182034221e-05


In [ ]:
import pickle

n_reps = 4

filepath = f"../lucj/lucj_initial_params/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/data.pickle"

with open(filepath, "rb") as f:
    data_ref = pickle.load(f)

data_type = ""
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data = pickle.load(f)

data_type = "asc_"
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data_acs = pickle.load(f)

data_type = "des_"
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data_des = pickle.load(f)

data_type = "rand_"
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data_rand = pickle.load(f)

print(data_ref)
print(data)
print(data_acs)
print(data_des)
print(data_rand)

{'energy': np.float64(-108.98202108022255), 'error': np.float64(0.06465069729802053), 'spin_squared': np.float64(8.240600677639622e-30), 'entropy': np.float64(0.6769253824916961), 'n_reps': 4}
{'energy': np.float64(-108.7514498549557), 'error': np.float64(0.29522192256487756), 'spin_squared': np.float64(1.4170653773900526e-29), 'entropy': np.float64(1.904546204865731)}
{'energy': np.float64(-108.80808128976219), 'error': np.float64(0.23859048775838687), 'spin_squared': np.float64(1.3767113217295944e-29), 'entropy': np.float64(1.864401083904107)}
{'energy': np.float64(-108.75679145243078), 'error': np.float64(0.289880325089797), 'spin_squared': np.float64(1.431655016029493e-29), 'entropy': np.float64(1.8726318761490581)}
{'energy': np.float64(-108.75443807695096), 'error': np.float64(0.29223370056961073), 'spin_squared': np.float64(1.3382386901810966e-29), 'entropy': np.float64(1.8913355719259273)}


In [9]:
import pickle

n_reps = 6

filepath = f"../lucj/lucj_initial_params/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/data.pickle"
with open(filepath, "rb") as f:
    data_ref = pickle.load(f)

data_type = ""
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data = pickle.load(f)

data_type = "asc_"
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data_acs = pickle.load(f)

data_type = "des_"
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data_des = pickle.load(f)

data_type = "rand_"
filepath = f"../lucj/n2_6-31g_10e16o/bond_distance-1.00000/connectivity-all-to-all/n_reps-{n_reps}/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/{data_type}data.pickle"
with open(filepath, "rb") as f:
    data_rand = pickle.load(f)

print(data_ref)
print(data)
print(data_acs)
print(data_des)
print(data_rand)

{'energy': np.float64(-108.9877860827798), 'error': np.float64(0.058885694740766326), 'spin_squared': np.float64(1.236701814148147e-29), 'entropy': np.float64(0.6965090899314161), 'n_reps': 6}
{'energy': np.float64(-108.76157524075484), 'error': np.float64(0.2850965367657352), 'spin_squared': np.float64(2.0371654898623102e-29), 'entropy': np.float64(1.8771199112447086)}
{'energy': np.float64(-108.76993317768303), 'error': np.float64(0.2767385998375431), 'spin_squared': np.float64(2.1286679512423744e-29), 'entropy': np.float64(1.8644891142283133)}
{'energy': np.float64(-108.7905509425483), 'error': np.float64(0.256120834972279), 'spin_squared': np.float64(2.129115038727032e-29), 'entropy': np.float64(1.8168608295545239)}
{'energy': np.float64(-108.7867676360835), 'error': np.float64(0.25990414143707596), 'spin_squared': np.float64(2.0714287490578974e-29), 'entropy': np.float64(1.841821218934379)}


In [3]:
import pickle

filepath = "../lucj/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-all-to-all/n_reps-5/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/opt_data.pickle"
with open(filepath, "rb") as f:
    data = pickle.load(f)

print(data)

{'init_loss': Array(0.00214424, dtype=float32), 'final_loss': Array(0.00026353, dtype=float32)}


In [11]:
import pickle

file = "../lucj/n2_sto-6g_10e8o/bond_distance-1.20000/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/hardware_sample.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

data = data[:10]

print(data)

BitArray(<shape=(), num_shots=6000, num_bits=16>)
BitArray(<shape=(), num_shots=10, num_bits=16>)


In [4]:
import pickle

file = "/media/storage/WanHsuan.Lin/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/shots-100000/samples_per_batch-1000/n_batches-3/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-100/symmetrize_spin-True/entropy-0/max_dim-500/hardware_sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

quimb_opt_energy_init = -109.0715272665909
quimb_opt_energy = -109.07862247270432
print(quimb_opt_energy_init - quimb_opt_energy)

{'energy': np.float64(-109.09370079472708), 'error': np.float64(0.004430025305794061), 'spin_squared': np.float64(0.0018572572652080501), 'sci_vec_shape': (500, 500)}
0.0070952061134192945


In [ ]:
import pickle

file = "/media/storage/WanHsuan.Lin/lucj_sqd_quimb/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/maxiter-25/shots-100000/samples_per_batch-1000/n_batches-3/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-100/symmetrize_spin-True/entropy-0/max_dim-500/max_bond-10/cutoff-0.001/perm_mps-True/seed-0/data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

 message: The callback requested to stop the optimization procedure
 success: True
  status: 3
     fun: -109.08006822647131
       x: [ 8.992e-01 -4.035e-01 ...  0.000e+00  0.000e+00]
     nit: 0
   maxcv: 0.0
    nfev: 4


In [ ]:
import pickle

file = "/media/storage/WanHsuan.Lin/fe2s2_30e20o/random_sample/valid_string_only/shots-100000/samples_per_batch-1000/n_batches-3/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-100/symmetrize_spin-True/entropy-0/max_dim-500/sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

{'energy': -116.40176190890885, 'error': 0.09683844623194204, 'spin_squared': np.float64(4.913885505353448), 'sci_vec_shape': (500, 500)}


In [1]:
import pickle

file = "/media/storage/WanHsuan.Lin/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-all-to-all/n_reps-100/with_final_orbital_rotation-True/truncated/shots-100000/samples_per_batch-4000/n_batches-10/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-1/symmetrize_spin-True/entropy-0/max_dim-4000/sqd_data.pickle"

with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

{'energy': np.float64(-109.09173272707557), 'error': np.float64(0.006398092957311974), 'spin_squared': np.float64(0.002349151101755644), 'sci_vec_shape': (274, 274), 'history_energy': [[np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557), np.float64(-109.09173272707557)]], 'history_sci_vec_shape': [[(274, 274), (274, 274), (274, 274), (274, 274), (274, 274), (274, 274), (274, 274), (274, 274), (274, 274), (274, 274)]]}


In [1]:
import pickle

file = "/media/storage/WanHsuan.Lin/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/shots-100000/samples_per_batch-4000/n_batches-10/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-1/symmetrize_spin-True/entropy-0/max_dim-4000/sqd_data.pickle"
file = "/media/storage/WanHsuan.Lin/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/shots-100000/samples_per_batch-4000/n_batches-3/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-100/symmetrize_spin-True/entropy-0/max_dim-4000/hardware_sqd_data.pickle"
file = "/media/storage/WanHsuan.Lin/n2_6-31g_10e16o/bond_distance-1.20000/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/truncated/shots-100000/samples_per_batch-4000/n_batches-3/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-100/symmetrize_spin-True/entropy-0/max_dim-4000/hardware_sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

quimb_energy = -109.06259827427564

{'energy': np.float64(-109.09725338156949), 'error': np.float64(0.0008774384633909449), 'spin_squared': np.float64(0.0004468365159652513), 'sci_vec_shape': (2361, 2361)}


In [ ]:
import pickle
import numpy as np

file = "/media/storage/WanHsuan.Lin/fe2s2_30e20o/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/shots-100000/samples_per_batch-4000/n_batches-10/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-1/symmetrize_spin-True/entropy-0/max_dim-4000/sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

history_energy = data["history_energy"]
print(np.min(history_energy))
print(np.max(history_energy))

quimb_energy = -116.02633567631842

dmrg_energy = -116.6056091

print(quimb_energy - dmrg_energy)
print(data["energy"] - dmrg_energy)

{'energy': -115.99817083380935, 'error': 0.5004295213314407, 'spin_squared': np.float64(4.295637625971987), 'sci_vec_shape': (929, 929), 'history_energy': [[-115.9914404382526, -115.9190156832594, -115.98523054711447, -115.95370093080437, -115.9584855455556, -115.96508767122359, -115.99790246066303, -115.99817083380935, -115.96307630785326, -115.93596585640309]], 'history_sci_vec_shape': [[(904, 904), (920, 920), (914, 914), (911, 911), (930, 930), (893, 893), (921, 921), (929, 929), (893, 893), (917, 917)]]}
-115.99817083380935
-115.9190156832594
0.5792734236815704
0.6074382661906412


In [1]:
import pickle
import numpy as np

file = "/media/storage/WanHsuan.Lin/fe2s2_30e20o/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/shots-100000/samples_per_batch-4000/n_batches-10/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-1/symmetrize_spin-True/entropy-0/max_dim-4000/sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

history_energy = data["history_energy"]
print(np.min(history_energy))
print(np.max(history_energy))

quimb_energy = -116.02633567631842

dmrg_energy = -116.6056091

print(quimb_energy - dmrg_energy)
print(data["energy"] - dmrg_energy)

{'energy': -115.99817083380935, 'error': 0.5004295213314407, 'spin_squared': np.float64(4.295637625971987), 'sci_vec_shape': (929, 929), 'history_energy': [[-115.9914404382526, -115.9190156832594, -115.98523054711447, -115.95370093080437, -115.9584855455556, -115.96508767122359, -115.99790246066303, -115.99817083380935, -115.96307630785326, -115.93596585640309]], 'history_sci_vec_shape': [[(904, 904), (920, 920), (914, 914), (911, 911), (930, 930), (893, 893), (921, 921), (929, 929), (893, 893), (917, 917)]]}
-115.99817083380935
-115.9190156832594
0.5792734236815704
0.6074382661906412


In [ ]:
import pickle
import numpy as np

file = "/media/storage/WanHsuan.Lin/fe2s2_30e20o/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/quimb/maxiter-25/shots-50000/samples_per_batch-4000/n_batches-10/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-1/symmetrize_spin-True/entropy-0/max_dim-4000/max_bond-100/cutoff-1e-10/perm_mps-False/seed-0/sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

file = "/media/storage/WanHsuan.Lin/fe2s2_30e20o/connectivity-heavy-hex/n_reps-1/with_final_orbital_rotation-True/multi_stage_optimization-True/begin_reps-20/step-2/quimb/maxiter-25/shots-10000/samples_per_batch-4000/n_batches-10/energy_tol-1e-05/occupancies_tol-0.001/carryover_threshold-0.001/max_iterations-1/symmetrize_spin-True/entropy-0/max_dim-4000/max_bond-100/cutoff-1e-10/perm_mps-False/seed-0/sqd_data.pickle"
with open(file, "rb") as f:
    data = pickle.load(f)
print(data)

{'energy': -116.05586221869093, 'error': 0.5461591064471065, 'spin_squared': np.float64(3.784865078126809), 'sci_vec_shape': (932, 932), 'history_energy': [[-116.01390479649743, -115.97461668275372, -116.00279912304097, -116.05586221869093, -116.05266024374241, -116.04944027272991, -116.03523228655342, -116.05515557569062, -116.04772135326193, -116.02697666982161]], 'history_sci_vec_shape': [[(902, 902), (913, 913), (929, 929), (932, 932), (919, 919), (927, 927), (897, 897), (917, 917), (918, 918), (939, 939)]]}
{'energy': -116.05336313868156, 'error': 0.5486581864564783, 'spin_squared': np.float64(3.99827495647734), 'sci_vec_shape': (1007, 1007), 'history_energy': [[-115.9684471973452, -115.96415744279983, -115.9449267282708, -116.01862050100735, -116.00460806608366, -116.05336313868156, -116.02029765220134, -116.0227461690876, -116.02633567624375, -116.0089715625363]], 'history_sci_vec_shape': [[(1024, 1024), (1021, 1021), (983, 983), (998, 998), (966, 966), (1007, 1007), (970, 970),